# Pseudonimización de Datos en Data Science

A veces, por diversos motivos, no es posible perder o distorsionar información relevante para los fines de la investigación, por más sensibles que sean esos datos. 😞

Sin embargo, existen otro conjunto de técnicas que, en lugar de hacer los datos anónimos, hace que podamos referirnos a ellos a través de **pseudónimos** o motes, lo cual también es efectivo para cuidar la privacidad de los datos, pero sin perderlos.

Es decir, las técnicas de pseudonimización reemplazan los datos reales por datos de fantasía que **enmascaran** a los datos que debemos proteger, permitiéndonos realizar todas las tareas de análisis de datos con esa información, pero sin revelar la verdadera identidad de dicha información.

Estas técnicas de pseudonimización, bien implementadas, están permitidas por las regulaciones legales en materia de protección de datos como la [GDPR](https://gdpr-info.eu/) y la [CCPA](https://oag.ca.gov/privacy/ccpa).

## Caso de ejemplo

In [7]:
import uuid
import hashlib
from typing import Any

import pandas as pd

In [2]:
names: list[str] = ['Ana', 'Juan', 'Luis']
emails: list[str] = ['ana@ejemplo.com', 'juan@ejemplo.com', 'luis@ejemplo.com']
cities: list[str] = ['Ciudad A', 'Ciudad B', 'Ciudad C']

data: dict[str, list[str]] = {
    'Nombre': names,
    'Email': emails,
    'Ubicación': cities
}

df: pd.DataFrame = pd.DataFrame(data)
df

,Nombre,Email,Ubicación
0,Ana,ana@ejemplo.com,Ciudad A
1,Juan,juan@ejemplo.com,Ciudad B
2,Luis,luis@ejemplo.com,Ciudad C


### 1. Reemplazar el nombre de la persona con un identificador único universal (UUID)

In [ ]:
# Generar los UUID para cada instancia
uuid_container: list[str] = []

for _ in range(len(df)):
    uuid_container.append(str(uuid.uuid4()))

uuid_container

['ffcc810c-bab8-46fc-a512-9a09ae234558',
 '1a31bc23-e80a-4085-84de-ffdc91e3bf8b',
 'aed2ef66-22a4-4c36-9303-e5e93226a3a5']

In [5]:
# Reemplazar la columna 'Nombre' con los UUID generados
df['UUID'] = uuid_container
df.drop(['Nombre'], axis=1, inplace=True)
df

,Email,Ubicación,UUID
0,ana@ejemplo.com,Ciudad A,ffcc810c-bab8-46fc-a512-9a09ae234558
1,juan@ejemplo.com,Ciudad B,1a31bc23-e80a-4085-84de-ffdc91e3bf8b
2,luis@ejemplo.com,Ciudad C,aed2ef66-22a4-4c36-9303-e5e93226a3a5


### 2. Hashing

Consiste en usar una función para convertir los valores de datos en una cadena de caracteres fija que reemplace a los valores originales. La diferencia con el UUID radica en que éste genera un valor completamente aleatorio sin depender del dato original, mientras que el hashing **genera un valor a partir del dato original** (es decir, que tiene una raíz en el valor original), pero distorsionándolo hasta que se hace irreconocible (al menos, a los humanos). De todos modos, ambas transformaciones son irreversibles.

In [ ]:
def hash_data(data: Any) -> str:
    """
    Return the SHA256 hash of the input data.
    """
    return hashlib.sha256(data.encode('utf-8')).hexdigest()

In [9]:
hashed_emails: list[str] = [hash_data(email) for email in df['Email']]
hashed_emails

['52f5655387a4bdcb850a93ce7223979b4a360fa146c89c47d5d897d10a13f76e',
 '37d52dabac00945d10d0879cad2564b4985b1a3303e4b5d571fe1f0f733bb66f',
 '59f7ac6c8ec549f037cd7188ca36e499099f041159cfd990f55df64f3af9b03b']

In [10]:
df['Email'] = hashed_emails
df

,Email,Ubicación,UUID
0,52f5655387a4bdcb850a93ce7223979b4a360fa146c89c...,Ciudad A,ffcc810c-bab8-46fc-a512-9a09ae234558
1,37d52dabac00945d10d0879cad2564b4985b1a3303e4b5...,Ciudad B,1a31bc23-e80a-4085-84de-ffdc91e3bf8b
2,59f7ac6c8ec549f037cd7188ca36e499099f041159cfd9...,Ciudad C,aed2ef66-22a4-4c36-9303-e5e93226a3a5


### Tokenización

Al igual que el hashing, transforma los valores sensibles en valores de fantasía irreconocibles, con la diferencia de que la tokenización **sí preserva los valores originales** y permite que el usuario investigador manipule los valores de fantasía, mientras que el programa internamente hace sus procesos con la información verdadera. Los valores de fantasía son llamados **tokens**.

Es una técnica ampliamente utilizada en casos donde la **información privada** simplemente **no puede ser eliminada** (por ejemplo, en la base de datos de un centro médico).

In [12]:
tokens: dict[str, str] = {}

In [13]:
def generate_token(data: Any) -> str:
    """
    Tokenize the input data.
    """
    token: str = str(uuid.uuid4())
    tokens[token] = data
    return token

In [14]:
def recover_data(token: str) -> Any:
    """
    Recover the original data from the token.
    """
    return tokens.get(token, 'Invalid token!')

In [15]:
original_data_example: str = '123-456-789'
token: str = generate_token(original_data_example)
token

'107d4f52-752a-4c68-9b88-a41f66b14dc4'

In [16]:
recovered_data_example: str = recover_data(token)
recovered_data_example

'123-456-789'